In [ ]:
# -----------------------------
# Chicken Pastil Production Calculator with GUI
# -----------------------------
# This program calculates production metrics (max units, cost, profit, break-even)
# based on ingredient availability, cost, and user-defined selling price and fixed cost.
# It includes a simple GUI using tkinter and generates a graph using matplotlib.

# --- Import necessary libraries ---
import tkinter as tk                            # For GUI
from tkinter import messagebox                  # For pop-up result messages
import math                                     # For ceiling function (break-even)
import matplotlib.pyplot as plt                 # For plotting graphs
from IPython.display import display, clear_output  # To manage output in Jupyter

# --- Step 1: Define ingredient data ---
# Each ingredient has available quantity, cost per unit (gram/ml), and recipe amount per unit product
ingredients = {
    "Rice": {"available": 10_000, "cost": 50 / 1000, "recipe": 150},
    "Chicken breast": {"available": 5_000, "cost": 200 / 1000, "recipe": 100},
    "Onion": {"available": 1_000, "cost": 60 / 1000, "recipe": 10},
    "Garlic": {"available": 500, "cost": 120 / 1000, "recipe": 5},
    "Soy Sauce": {"available": 1000, "cost": 30 / 1000, "recipe": 10},
    "Cooking Oil": {"available": 1000, "cost": 80 / 1000, "recipe": 5},
    "Banana Leaf": {"available": 500, "cost": 25 / 1000, "recipe": 20},
}

# --- Step 2: Define the calculation function (runs when "Calculate" is clicked) ---
def calculate():
    try:
        clear_output(wait=True)  # Clears previous output (useful in Jupyter)
        
        # --- Step 2.1: Get user inputs ---
        price = float(entry_price.get())      # Selling price per unit
        fixed_cost = float(entry_fixed.get()) # Total fixed overhead cost

        # --- Step 2.2: Compute max producible units based on most limiting ingredient ---
        max_units = min(
            ingredients[name]["available"] // ingredients[name]["recipe"]
            for name in ingredients
        )

        # --- Step 2.3: Compute total variable cost for max units ---
        total_cost = sum(
            ingredients[name]["recipe"] * ingredients[name]["cost"] * max_units
            for name in ingredients
        )

        # --- Step 2.4: Compute cost per unit, revenue, profit, and break-even units ---
        cost_per_unit = total_cost / max_units if max_units > 0 else 0
        revenue = max_units * price
        profit = revenue - total_cost
        breakeven = math.ceil(fixed_cost / (price - cost_per_unit)) if price > cost_per_unit else float('inf')

        # --- Step 2.5: Prepare and show results as a pop-up message ---
        output_text = (
            f"Maximum producible units: {int(max_units)}\n"
            f"Total cost: ₱{total_cost:.2f}\n"
            f"Cost per unit: ₱{cost_per_unit:.2f}\n"
            f"Revenue: ₱{revenue:.2f}\n"
            f"Profit: ₱{profit:.2f}\n"
            f"Break-even units: {breakeven if breakeven != float('inf') else 'N/A (No profit)'}"
        )
        messagebox.showinfo("Results", output_text)

        # --- Step 2.6: Generate graph of cost, revenue, and profit over units produced ---
        units = list(range(0, int(max_units) + 5))
        costs = [sum(ingredients[name]["recipe"] * ingredients[name]["cost"] * u for name in ingredients) for u in units]
        revenues = [u * price for u in units]
        profits = [rev - cost for rev, cost in zip(revenues, costs)]

        plt.figure(figsize=(10, 5))
        plt.plot(units, costs, label="Total Cost", color='blue')
        plt.plot(units, revenues, label="Revenue", color='green')
        plt.plot(units, profits, label="Profit", color='orange')
        plt.axhline(y=fixed_cost, color='red', linestyle='--', label='Fixed Cost')
        plt.axvline(x=breakeven if breakeven != float('inf') else 0, color='gray', linestyle=':', label='Break-even Units')

        plt.title("Chicken Pastil Production Metrics")
        plt.xlabel("Units Produced")
        plt.ylabel("₱ (Peso)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    except ValueError:
        # Handles invalid (non-numeric) input
        messagebox.showerror("Input Error", "Please enter valid numeric values.")

# --- Step 3: Create GUI layout ---
root = tk.Tk()
root.title("Chicken Pastil Production Calculator")

# Input fields for selling price and fixed overhead
tk.Label(root, text="Selling Price per Unit (₱):").grid(row=0, column=0)
entry_price = tk.Entry(root)
entry_price.grid(row=0, column=1)

tk.Label(root, text="Fixed Overhead Cost (₱):").grid(row=1, column=0)
entry_fixed = tk.Entry(root)
entry_fixed.grid(row=1, column=1)

# Button to trigger calculation
tk.Button(root, text="Calculate", command=calculate).grid(row=2, column=0, columnspan=2, pady=10)

# Start the GUI application
root.mainloop()
